In [8]:
import requests
import json
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np

#helper function to get responses of the api and store each layer in a dictionary
def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

In [9]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [10]:
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import branca

#dynamic legend values
column_names = ['low_edu_lv', 'unemp_peop', 'pop_dens', 'gender_r', 'you_peop_r', 'eld_peop_r']

data = {column_name: [] for column_name in column_names}

responses = [
    get_responses('lisbon_low_literacy_level_ratio')['lisbon_low_literacy_level_ratio']['features'],
    get_responses('lisbon_unemployed_people_ratio')['lisbon_unemployed_people_ratio']['features'],
    get_responses('lisbon_population_density')['lisbon_population_density']['features'],
    get_responses('lisbon_gender_ratio')['lisbon_gender_ratio']['features'],
    get_responses('lisbon_youth_people_ratio')['lisbon_youth_people_ratio']['features'],
    get_responses('lisbon_elderly_people_ratio')['lisbon_elderly_people_ratio']['features']
]

for response_data in responses:
    for feature in response_data:
        properties = feature['properties']
        for column_name in column_names:
            value = properties.get(column_name, None)
            data[column_name].append(value)
        
df = pd.DataFrame(data)

#print(df)

min_values = df.min(skipna=True)
max_values = df.max(skipna=True)

num_groups = 4
group_boundaries = {}
for column in df.columns:
    min_value = min_values[column]
    max_value = max_values[column]
    
    if min_value is None or max_value is None:
        continue  # Skip columns with missing or null values
        
    group_size = (max_value - min_value) / num_groups
    boundaries = np.arange(min_value, max_value + group_size, group_size)
    group_boundaries[column] = boundaries

# Print the group boundaries for each column
for column, boundaries in group_boundaries.items():
    print(f"Group boundaries for column '{column}':")
    print(boundaries)
    print()

Group boundaries for column 'low_edu_lv':
[  0.  25.  50.  75. 100.]

Group boundaries for column 'unemp_peop':
[  0.  25.  50.  75. 100.]

Group boundaries for column 'pop_dens':
[  0.         167.09601145 334.19202289 501.28803434 668.38404579]

Group boundaries for column 'gender_r':
[  0. 225. 450. 675. 900.]

Group boundaries for column 'you_peop_r':
[  0.  25.  50.  75. 100.]

Group boundaries for column 'eld_peop_r':
[  0.  25.  50.  75. 100.]



In [11]:
def hexmap(responses, column_name, color, feature_group):
    for response in responses:
        for feature in response:
            properties = feature['properties']
            if column_name in properties:
                value = properties[column_name]
                geoj = folium.GeoJson(feature, style_function=color)
                folium.features.GeoJsonPopup(fields=[column_name], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
                feature_group.add_child(geoj)
                

#ppo_low_ed

color_lowed = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['low_edu_lv'] < 0 else 
    '#E5E5E5' if 0 <= x['properties']['low_edu_lv'] < 25 else 
    '#CCCCCC' if 25 <= x['properties']['low_edu_lv'] < 50 else 
    '#999999' if 50 <= x['properties']['low_edu_lv'] < 75 else
    '#666666', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg1 = folium.FeatureGroup(name='People with low literacy level ratio',overlay=True, control=False)
hexmap(responses, 'low_edu_lv', color_lowed, fg1)
scale_fg1 = branca.colormap.StepColormap(['#E5E5E5','#CCCCCC','#999999','#666666'],index=[0,25,50,75,100], vmin=0, vmax=100, caption='People with low literacy level ratio')

#ppop_unemp

color_unemp = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['unemp_peop'] < 0 else 
    '#E5E5E5' if 0 <= x['properties']['unemp_peop'] < 25 else 
    '#CCCCCC' if 25 <= x['properties']['unemp_peop'] < 50 else 
    '#999999' if 50 <= x['properties']['unemp_peop'] < 75 else
    '#666666', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg2 = folium.FeatureGroup(name='Unemployed people ratio', overlay=True, control=False)
hexmap(responses, 'unemp_peop', color_unemp, fg2)
scale_fg2 = branca.colormap.StepColormap(['#E5E5E5','#CCCCCC','#999999','#666666'],index=[0,25,50,75,100], vmin=0, vmax=100, caption='Unemployed people ratio')

#pop_densit

color_densit = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pop_dens'] < 0 else 
    '#FFFF99' if 0 <= x['properties']['pop_dens'] < 167.10 else 
    '#FF6347' if 167.10 <= x['properties']['pop_dens'] < 334.19 else 
    '#AA33AA' if 334.19 <= x['properties']['pop_dens'] < 501.29 else
    '#4B0082', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg3 = folium.FeatureGroup(name='Population density', overlay=True, control=False)
hexmap(responses, 'pop_dens', color_densit, fg3)
scale_fg3 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082'],index=[0, 167.10,334.19,501.29,668.38], vmin=0, vmax=668.38, caption='Population density')

#gender_rat

color_gender = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['gender_r'] < 0 else 
    '#FF6666' if 0 <= x['properties']['gender_r'] < 225 else 
    '#FFCCCC' if 225 <= x['properties']['gender_r'] < 450 else 
    '#CCDDFF' if 450 <= x['properties']['gender_r'] < 675 else
    '#87CEEB', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg4 = folium.FeatureGroup(name='Gender ratio',overlay=True, control=False)
hexmap(responses, 'gender_r', color_gender, fg4)
scale_fg4 = branca.colormap.StepColormap(['#FF6666','#FFCCCC','#CCDDFF','#87CEEB'],index=[0,225,450,675,900], vmin=0, vmax=900, caption='Gender ratio')

#yt_peop_ra

color_yt = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['you_peop_r'] < 0 else 
    '#FFEDA0' if 0 <= x['properties']['you_peop_r'] < 25 else 
    '#FFC100' if 25 <= x['properties']['you_peop_r'] < 50 else 
    '#FF8400' if 50 <= x['properties']['you_peop_r'] < 75 else
    '#FF4500', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg5 = folium.FeatureGroup(name='Youth people ratio',overlay=True, control=False)
hexmap(responses, 'you_peop_r', color_yt, fg5)
scale_fg5 = branca.colormap.StepColormap(['#FFEDA0','#FFC100','#FF8400','#FF4500'],index=[0,25,50,75,100], vmin=0, vmax=100, caption='Youth people ratio')

#ed_peop_ra

color_ed = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['eld_peop_r'] < 0 else 
    '#FF0000' if 0 <= x['properties']['eld_peop_r'] < 25 else 
    '#00FF00' if 25 <= x['properties']['eld_peop_r'] < 50 else 
    '#0000FF' if 50 <= x['properties']['eld_peop_r'] < 75 else
    '#FFFF00', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg6 = folium.FeatureGroup(name='Elderly people ratio',overlay=True, control=False)
hexmap(responses, 'eld_peop_r', color_ed, fg6)
scale_fg6 = branca.colormap.StepColormap(['#FF0000','#00FF00','#0000FF','#FFFF00'],index=[0,25,50,75,100], vmin=0, vmax=100, caption='Elderly people ratio')

In [12]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 7):
    # Create a new map object for each iteration
    m = folium.Map(location=[38.736946, -9.142685], control_scale=True, zoom_start=12, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'Output_Lisbon_Socio{i}.html'
    m.save(map_fname)